In [80]:
import pandas as pd
import scipy.sparse
import scipy
import scipy.io
import numpy as np
from sklearn.decomposition import PCA
from sklearn.utils import resample
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.manifold import TSNE
import warnings
warnings.filterwarnings('ignore')

In [3]:
train_clean = pd.read_csv('Data/train_clean.tsv', sep='\t', header=0)

In [48]:
#get the mean values per category_name_1 to use as baseline predictions
mean_values = train_clean.groupby('category_name_1')['price'].mean()
df_means = pd.DataFrame(mean_values)
df_means.reset_index(inplace=True)
df_means.rename(columns={'price':'prediction'}, inplace=True)

In [73]:
testFile = pd.read_csv('Data/test.tsv', sep='\t', header=0)

#split by category_name
#df.join(df['AB'].str.split('-', 1, expand=True).rename(columns={0:'A', 1:'B'}))
#testFile['category_name'].str.split('/',2, expand=True).rename(columns={0:'category_1', 1:'category_2', 2:'category_3'})

In [77]:
#splitting train_clean into train_base and test_base as our test.tsv has no price labels for us to check against
train_base = train_clean.iloc[:1000000,:]
test_base = train_clean.iloc[1000000:,:]

In [82]:
#import train_term_matrix
sparse_matrix = scipy.io.mmread('Data/train_term_matrix.mtx')

#dies
#train_dense = sparse_matrix.todense()

In [ ]:
train_dense.columns

## Baseline Model - Use average of category_name_1 products

In [78]:
from sklearn.metrics import mean_squared_error

#merge with df_means to get mean 'prediction'
test_base_pred = test_base.merge(df_means, how='left', on='category_name_1')

#remove rows with NaN in category_1
test_base_pred = test_base_pred[test_base_pred.category_name_1.notnull()]

mse_base = mean_squared_error(test_base_pred['price'], test_base_pred['prediction'])
print('Baseline model MSE = {:.2f}'.format(mse_base))


Baseline model MSE = 1450.86


## Regression Tree

>> need help how to handle (subset/sample) the sparse matrix, as our test data doesn't have a price, I think we should split our train into train & test(or validation)

>> need to set XTrain, XTest, yTrain, yTest

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

treeReg = DecisionTreeRegressor(random_state=0)

params_maxDepth = {'max_depth':[2,5,10,15,20]}

clf = GridSearchCV(treeReg, params_maxDepth)
clf.fit(XTrain, yTrain)

In [75]:
## Random Forest Regression

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state=0)

clf_rf = GridSearchCV(rf, params_maxDepth)
clf.fit(XTrain, yTrain)

## Support Vector Regression (SVR)

In [ ]:
from sklearn.svm import SVR

svrRBF = SVR(kernel='rbf', C=1000, gamma=0.1)
svrLIN = SVR(kernel='linear', C=1000)
svrPOLY = SVR(kernel='poly', C=1000, degree=3)

y_rbf = svr_rbf.fit(XTrain, yTrain).predict(XTest)
y_lin = svr_lin.fit(XTrain, yTrain).predict(XTest)
y_poly = svr_poly.fit(XTrain, yTrain).predict(XTest)

## Regression Models that take sparse matrices (
DecisionTreeRegressor
BaggingRegressor
RandomForestRegressor
ExtraTreesRegressor
AdaBoostRegressor
KNeighborsRegressor
SVR
SGDRegressor

Linear Regression
Ridge Regression
Lasso Regression
ElasticNet
PassiveAggressiveRegressor
RANSACRegressor
KernelRidge
NuSVR
LinearSVR
RadiusNeighborsRegressor
MultiOutputRegressor
MLPRegressor